# Taller 3

https://r2022.netlify.app/

In [1]:
install.packages("readxl")
install.packages("readr")
install.packages("stringr")
install.packages("dplyr")
install.packages("sqldf")
install.packages("utils")
install.packages("data.table")
library(data.table)
library(readxl)
library(readr)
library(stringr)
library(dplyr)
library(sqldf)
library(utils)

Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”
Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Warning message in install.packages("readr"):
“installation of package ‘readr’ had non-zero exit status”
Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Warning message in install.packages("dplyr"):
“installation of package ‘dplyr’ had non-zero exit status”
Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘utils’ is a base package, and should not be updated”
Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Warning message in install.packages("data.table"):
“installation of package ‘data.table’ had non-zero exit status

## IMDb (Internet Movie Database), Punto 2

In [2]:
createDf <- function(name){
  base <- "/datasets/taller3/CE/"
  df <- read.delim(gzfile(str_c(base,name)), sep="\t")
  return(df)
}

In [3]:
df_TitleBasics <- createDf("title.basics.tsv.gz")
df_NameBasics <- createDf("name.basics.tsv.gz")
df_TitlePrincipals <- createDf("title.principals.tsv.gz")
df_TitleRatings <- createDf("title.ratings.tsv.gz")

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”


In [4]:
head(df_TitleBasics, n = 3)
head(df_NameBasics, n = 3)
head(df_TitlePrincipals, n = 3)
head(df_TitleRatings, n = 3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
2,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
3,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0045537,tt0053137,tt0072308,tt0050419"
2,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0038355,tt0071877"
3,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0057345,tt0056404,tt0054452"


,tconst,ordering,nconst,category,job,characters
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,tt0000001,1,nm1588970,self,\N,[Self]
2,tt0000001,2,nm0005690,director,\N,\N
3,tt0000001,3,nm0374658,cinematographer,director of photography,\N


,tconst,averageRating,numVotes
,<chr>,<dbl>,<int>
1,tt0000001,5.7,1957
2,tt0000002,5.8,263
3,tt0000003,6.5,1791


### Cuantas películas realizadas en la década de los 80 tienen títulos que inician y/o terminan con la palabra "Aliens"?

In [5]:
#4 peliculas
r1 <- sqldf("SELECT *
             FROM df_TitleBasics
             WHERE startYear LIKE '198_%' AND 
                   titleType IN ('tvMovie','movie') AND
                   (primaryTitle LIKE 'Aliens_%' OR 
                   primaryTitle LIKE '% Aliens')")

r1
nrow(r1)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
tt0083538,tvMovie,Aliens from Another Planet,Aliens from Another Planet,0,1982,\N,96,Sci-Fi
tt0095431,movie,Katy Meets the Aliens,"Katy, Kiki y Koko",0,1988,\N,70,Animation
tt0096402,movie,Voyage of the Rock Aliens,Voyage of the Rock Aliens,0,1984,\N,97,"Comedy,Musical,Sci-Fi"
tt0173628,movie,"Aliens, Dragons, Monsters and Me","Aliens, Dragons, Monsters, & Me",0,1983,\N,60,Documentary


[1] 4

###  Cual de ellas tiene la mayor duración?

In [6]:
# Voyage of the Rock Aliens - 97 minutos
r1a <- sqldf("SELECT *
             FROM r1
             WHERE runtimeMinutes NOT IN ('\\N')
             ORDER BY runtimeMinutes DESC")

head(r1a, n=1)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,tt0096402,movie,Voyage of the Rock Aliens,Voyage of the Rock Aliens,0,1984,\N,97,"Comedy,Musical,Sci-Fi"


### Quien dirigió esta película?

In [7]:
#Dirigido por James Fargo
r1b <- sqldf("SELECT nconst,category,primaryTitle
             FROM r1a INNER JOIN df_TitlePrincipals ON (df_TitlePrincipals.tconst=r1a.tconst)
             WHERE primaryTitle LIKE 'Voyage of the Rock Aliens' and category LIKE 'director'
")
r1c <- sqldf("SELECT *
             FROM r1b INNER JOIN df_NameBasics ON (df_NameBasics.nconst=r1b.nconst)
")
r1c

nconst,category,primaryTitle,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
nm0267309,director,Voyage of the Rock Aliens,nm0267309,James Fargo,1938,\N,"director,assistant_director,producer","tt0074483,tt0077523,tt5421618,tt0073195"


###  Cual es la película mejor calificada de este(a) director(a)?

In [8]:
#Aliens, Dragons, Monsters and Me 7.6
r1e <- sqldf("SELECT r1.*,averageRating
             FROM r1 INNER JOIN df_TitleRatings ON (r1.tconst=df_TitleRatings.tconst)
             ORDER BY averageRating	 DESC")
head(r1e)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,tt0173628,movie,"Aliens, Dragons, Monsters and Me","Aliens, Dragons, Monsters, & Me",0,1983,\N,60,Documentary,7.6
2,tt0095431,movie,Katy Meets the Aliens,"Katy, Kiki y Koko",0,1988,\N,70,Animation,6.5
3,tt0096402,movie,Voyage of the Rock Aliens,Voyage of the Rock Aliens,0,1984,\N,97,"Comedy,Musical,Sci-Fi",5.1
4,tt0083538,tvMovie,Aliens from Another Planet,Aliens from Another Planet,0,1982,\N,96,Sci-Fi,5.0


### Cuantas películas han sido dirigidas por "Werner Herzog"? 

In [ ]:
#98
t1a <- sqldf("SELECT *
             FROM df_TitleBasics INNER JOIN df_TitlePrincipals ON (df_TitlePrincipals.tconst=df_TitleBasics.tconst)
             WHERE nconst IN (SELECT nconst
                              FROM df_NameBasics
                              WHERE primaryName LIKE 'Werner Herzog') AND
                              titleType IN ('tvMovie','movie')")
#nm0001348
t1a
nrow(t1a)

KernelInterrupted: Execution interrupted by the Jupyter kernel.

### Cual de ellas tiene la mayor calificación? 

In [1]:
#
t1b <- sqldf("SELECT t1a.*,averageRating
             FROM t1a INNER JOIN df_TitleRatings ON (t1a.tconst=df_TitleRatings.tconst)
             ORDER BY averageRating DESC")
head(t1b)

ERROR: Error in sqldf("SELECT t1a.*,averageRating\n             FROM t1a INNER JOIN df_TitleRatings ON (t1a.tconst=df_TitleRatings.tconst)\n             ORDER BY averageRating DESC"): could not find function "sqldf"


### En cuales de ellas participa "Klaus Kinski" como actor? 

In [ ]:
t1c <- sqldf("SELECT t1a*, primaryName
             FROM t1a INNER JOIN df_NameBasics ON (t1a.nconst=df_NameBasics.nconst)
             WHERE primaryName LIKE 'Klaus Kinski'
")
t1c

### Cuales actores/actrices, además de "Klaus Kinski", participaron en la película titulada "Nosferatu the Vampyre"?

In [ ]:
t1d <- sqldf("SELECT *
              FROM df_NameBasics INNER JOIN df_TitlePrincipals  ON (df_NameBasics.nconst=df_TitlePrincipals.nconst)
              WHERE tconst IN (SELECT tconst
                               FROM df_TitleBasics 
                               WHERE primaryTitle LIKE 'Nosferatu the Vampyre' )")
t1d

### Cuantas películas han sido dirigidas por "Martin Scorsese"? 

In [ ]:
t1e <- sqldf("SELECT *
              FROM df_NameBasics INNER JOIN df_TitlePrincipals ON (df_TitlePrincipals.nconst=df_NameBasics.nconst)
              WHERE primaryName LIKE 'Martin Scorsese' AND category LIKE 'director'
")

t1f <- sqldf("SELECT t1e.*, primaryTitle, runtimeMinutes	
              FROM t1e INNER JOIN df_TitleBasics ON (t1e.tconst=df_TitleBasics.tconst)
              WHERE titleType IN ('tvMovie','movie')
              ORDER BY runtimeMinutes DESC
")
nrow(t1f)

### Cual de ellas tiene la mayor duración?

In [ ]:
head(t1f)

### En cuales de ellas participan "Joe Pesci" y "Robert De Niro" como actores?

In [ ]:
t1g <- sqldf("SELECT *
              FROM df_NameBasics INNER JOIN df_TitlePrincipals ON (df_TitlePrincipals.nconst=df_NameBasics.nconst)
              WHERE primaryName LIKE 'Martin Scorsese' AND category LIKE 'director'
")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cd2fca3f-dc64-4c71-8bd4-adda531fa911' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>